# NBA Player Performance Dynamics: Player-Team Fit Analysis

This notebook analyzes how players perform across different team systems, quantifies player adaptability, and identifies optimal style environments for different player types.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances

# Add the project root to the path so we can import our modules
sys.path.append('..')

# Import our modules
from src.data_processing import calculate_player_team_fit
from src.visualization import create_player_fit_cards
from src.utils import setup_plotting_style

# Set up plotting style
setup_plotting_style()

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## Load Processed Data

Let's load the processed data from previous notebooks.

In [ ]:
# Load the processed data
try:
    player_dynamics = pd.read_csv('../data/processed/player_dynamics.csv')
    team_styles = pd.read_csv('../data/processed/team_styles.csv')
    player_temporal_df = pd.read_csv('../data/processed/player_temporal.csv')
    games_processed = pd.read_csv('../data/processed/games_processed.csv')
    
    # Convert date strings to datetime objects
    player_temporal_df['GAME_DATE'] = pd.to_datetime(player_temporal_df['GAME_DATE'])
    games_processed['GAME_DATE'] = pd.to_datetime(games_processed['GAME_DATE'])
    
    print(f"Loaded player dynamics data with {len(player_dynamics)} players")
    print(f"Loaded team styles data with {len(team_styles)} teams")
    print(f"Loaded player temporal data with {len(player_temporal_df)} records")
    print(f"Loaded processed games data with {len(games_processed)} records")
except FileNotFoundError:
    print("Processed data not found. Please run the previous notebooks first.")

In [ ]:
# Examine the player dynamics data
player_dynamics.head()

In [ ]:
# Examine the team styles data
team_styles.head()

## Player-Team Style Compatibility

Let's analyze how players perform across different team styles.

In [ ]:
# Merge player game data with team style information
# First, add team style to games data
games_with_style = pd.merge(
    games_processed,
    team_styles[['Team_ID', 'style_name']],
    on='Team_ID',
    how='left'
)

# Then, merge with player game data
player_games_with_style = pd.merge(
    player_temporal_df,
    games_with_style[['Game_ID', 'style_name']],
    on='Game_ID',
    how='left'
)

# Check the merged data
print(f"Player games with style data: {len(player_games_with_style)} records")
player_games_with_style.head()

In [ ]:
# Calculate player performance metrics by team style
player_style_performance = player_games_with_style.groupby(['Player_ID', 'PlayerName', 'style_name']).agg({
    'PTS': ['mean', 'std'],
    'PLUS_MINUS': ['mean', 'std'],
    'Game_ID': 'count'
}).reset_index()

# Flatten the column names
player_style_performance.columns = ['_'.join(col).strip('_') for col in player_style_performance.columns.values]

# Rename some columns for clarity
player_style_performance = player_style_performance.rename(columns={
    'Game_ID_count': 'games_played'
})

# Filter players with enough games in each style
min_games = 5
player_style_performance = player_style_performance[player_style_performance['games_played'] >= min_games]

# Display the performance data
player_style_performance.head()

In [ ]:
# Calculate performance differences across styles for each player
player_style_diff = []

for player_id in player_style_performance['Player_ID'].unique():
    player_data = player_style_performance[player_style_performance['Player_ID'] == player_id]
    player_name = player_data['PlayerName'].iloc[0]
    
    # Calculate max-min differences
    pts_diff = player_data['PTS_mean'].max() - player_data['PTS_mean'].min()
    pm_diff = player_data['PLUS_MINUS_mean'].max() - player_data['PLUS_MINUS_mean'].min()
    
    # Calculate coefficient of variation across styles
    pts_cv = player_data['PTS_mean'].std() / player_data['PTS_mean'].mean() if player_data['PTS_mean'].mean() > 0 else 0
    pm_cv = player_data['PLUS_MINUS_mean'].std() / abs(player_data['PLUS_MINUS_mean'].mean()) if abs(player_data['PLUS_MINUS_mean'].mean()) > 0 else 0
    
    # Identify best and worst styles
    best_style_pts = player_data.loc[player_data['PTS_mean'].idxmax(), 'style_name']
    worst_style_pts = player_data.loc[player_data['PTS_mean'].idxmin(), 'style_name']
    best_style_pm = player_data.loc[player_data['PLUS_MINUS_mean'].idxmax(), 'style_name']
    worst_style_pm = player_data.loc[player_data['PLUS_MINUS_mean'].idxmin(), 'style_name']
    
    # Calculate adaptability score (inverse of average CV)
    adaptability_score = 1 / ((pts_cv + pm_cv) / 2 + 0.1)  # Add 0.1 to avoid division by zero
    
    # Add to results
    player_style_diff.append({
        'player_id': player_id,
        'player_name': player_name,
        'pts_diff': pts_diff,
        'pm_diff': pm_diff,
        'pts_cv': pts_cv,
        'pm_cv': pm_cv,
        'best_style_pts': best_style_pts,
        'worst_style_pts': worst_style_pts,
        'best_style_pm': best_style_pm,
        'worst_style_pm': worst_style_pm,
        'adaptability_score': adaptability_score
    })

# Convert to dataframe
player_adaptability = pd.DataFrame(player_style_diff)

# Sort by adaptability score
player_adaptability = player_adaptability.sort_values('adaptability_score', ascending=False)

# Display the adaptability data
player_adaptability.head()

In [ ]:
# Visualize player adaptability distribution
plt.figure(figsize=(10, 6))
plt.hist(player_adaptability['adaptability_score'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
plt.xlabel('Adaptability Score (Higher = More Adaptable)', fontsize=12)
plt.ylabel('Number of Players', fontsize=12)
plt.title('Distribution of Player Adaptability', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Identify most and least adaptable players
most_adaptable = player_adaptability.nlargest(10, 'adaptability_score')
least_adaptable = player_adaptability.nsmallest(10, 'adaptability_score')

print("Most Adaptable Players:")
print(most_adaptable[['player_name', 'adaptability_score', 'pts_cv', 'pm_cv', 'best_style_pm']])

print("\nLeast Adaptable Players:")
print(least_adaptable[['player_name', 'adaptability_score', 'pts_cv', 'pm_cv', 'best_style_pm']])

### Visualizing Player Performance Across Styles

Let's visualize how specific players perform across different team styles.

In [ ]:
# Select a sample of players with different adaptability levels
sample_players = [
    most_adaptable['player_name'].iloc[0],  # Most adaptable player
    least_adaptable['player_name'].iloc[0],  # Least adaptable player
    player_adaptability['player_name'].iloc[len(player_adaptability) // 2]  # Median adaptable player
]

# Create a figure with subplots
fig, axes = plt.subplots(len(sample_players), 2, figsize=(14, 4 * len(sample_players)))

# Plot performance across styles for each player
for i, player_name in enumerate(sample_players):
    # Get player data
    player_id = player_adaptability[player_adaptability['player_name'] == player_name]['player_id'].iloc[0]
    player_data = player_style_performance[player_style_performance['Player_ID'] == player_id]
    
    # Sort by style name
    player_data = player_data.sort_values('style_name')
    
    # Plot points by style
    axes[i, 0].bar(player_data['style_name'], player_data['PTS_mean'], yerr=player_data['PTS_std'], 
                  capsize=5, color='skyblue')
    axes[i, 0].set_title(f"{player_name}: Points by Style", fontsize=12)
    axes[i, 0].set_ylabel('Points Per Game', fontsize=10)
    axes[i, 0].tick_params(axis='x', rotation=45)
    axes[i, 0].grid(True, alpha=0.3)
    
    # Plot plus/minus by style
    axes[i, 1].bar(player_data['style_name'], player_data['PLUS_MINUS_mean'], yerr=player_data['PLUS_MINUS_std'], 
                  capsize=5, color='lightgreen')
    axes[i, 1].set_title(f"{player_name}: Plus/Minus by Style", fontsize=12)
    axes[i, 1].set_ylabel('Plus/Minus', fontsize=10)
    axes[i, 1].tick_params(axis='x', rotation=45)
    axes[i, 1].grid(True, alpha=0.3)
    axes[i, 1].axhline(y=0, color='gray', linestyle='--')
    
    # Add adaptability score to title
    adaptability = player_adaptability[player_adaptability['player_name'] == player_name]['adaptability_score'].iloc[0]
    axes[i, 0].set_title(f"{player_name}: Points by Style (Adaptability: {adaptability:.2f})", fontsize=12)

plt.tight_layout()
plt.show()

## Style Gap Analysis

Let's quantify performance differences across styles and identify style specialists vs. versatile players.

In [ ]:
# Merge player adaptability with player dynamics
player_full_analysis = pd.merge(
    player_dynamics,
    player_adaptability,
    on=['player_id', 'player_name'],
    how='inner'
)

# Display the merged data
player_full_analysis.head()

In [ ]:
# Analyze relationship between stability and adaptability
plt.figure(figsize=(10, 8))
plt.scatter(player_full_analysis['system_stability'], player_full_analysis['adaptability_score'], 
            alpha=0.7, c=player_full_analysis['avg_plus_minus'], cmap='RdYlGn')

# Add labels for notable players
for i, row in player_full_analysis.nlargest(5, 'adaptability_score').iterrows():
    plt.annotate(row['player_name'], 
                 (row['system_stability'], row['adaptability_score']),
                 fontsize=9)
    
for i, row in player_full_analysis.nsmallest(5, 'adaptability_score').iterrows():
    plt.annotate(row['player_name'], 
                 (row['system_stability'], row['adaptability_score']),
                 fontsize=9)

plt.xlabel('System Stability (lower = more stable)', fontsize=12)
plt.ylabel('Adaptability Score (higher = more adaptable)', fontsize=12)
plt.title('Player Stability vs. Adaptability', fontsize=14)
plt.colorbar(label='Plus/Minus')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate correlation between stability and adaptability
stability_adaptability_corr = player_full_analysis['system_stability'].corr(player_full_analysis['adaptability_score'])
print(f"Correlation between stability and adaptability: {stability_adaptability_corr:.3f}")

In [ ]:
# Classify players based on stability and adaptability
stability_threshold = player_full_analysis['system_stability'].median()
adaptability_threshold = player_full_analysis['adaptability_score'].median()

player_full_analysis['versatility_type'] = 'Unknown'
player_full_analysis.loc[(player_full_analysis['system_stability'] < stability_threshold) & 
                         (player_full_analysis['adaptability_score'] > adaptability_threshold), 'versatility_type'] = 'Stable Versatile'
player_full_analysis.loc[(player_full_analysis['system_stability'] >= stability_threshold) & 
                         (player_full_analysis['adaptability_score'] > adaptability_threshold), 'versatility_type'] = 'Volatile Versatile'
player_full_analysis.loc[(player_full_analysis['system_stability'] < stability_threshold) & 
                         (player_full_analysis['adaptability_score'] <= adaptability_threshold), 'versatility_type'] = 'Stable Specialist'
player_full_analysis.loc[(player_full_analysis['system_stability'] >= stability_threshold) & 
                         (player_full_analysis['adaptability_score'] <= adaptability_threshold), 'versatility_type'] = 'Volatile Specialist'

# Count players in each versatility type
versatility_counts = player_full_analysis['versatility_type'].value_counts().reset_index()
versatility_counts.columns = ['Versatility Type', 'Count']
versatility_counts['Percentage'] = versatility_counts['Count'] / len(player_full_analysis) * 100

versatility_counts

In [ ]:
# Visualize versatility types
plt.figure(figsize=(10, 8))

# Define colors for each versatility type
colors = {
    'Stable Versatile': 'green',
    'Volatile Versatile': 'orange',
    'Stable Specialist': 'blue',
    'Volatile Specialist': 'red'
}

# Plot each versatility type with a different color
for vtype, color in colors.items():
    type_data = player_full_analysis[player_full_analysis['versatility_type'] == vtype]
    plt.scatter(type_data['system_stability'], type_data['adaptability_score'], 
                label=vtype, color=color, alpha=0.7)

# Add quadrant lines
plt.axvline(x=stability_threshold, color='gray', linestyle='--')
plt.axhline(y=adaptability_threshold, color='gray', linestyle='--')

plt.xlabel('System Stability (lower = more stable)', fontsize=12)
plt.ylabel('Adaptability Score (higher = more adaptable)', fontsize=12)
plt.title('Player Versatility Types', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Compare performance metrics across versatility types
versatility_stats = player_full_analysis.groupby('versatility_type').agg({
    'avg_pts': 'mean',
    'avg_plus_minus': 'mean',
    'system_stability': 'mean',
    'adaptability_score': 'mean',
    'player_id': 'count'
}).reset_index()

# Rename count column
versatility_stats = versatility_stats.rename(columns={'player_id': 'player_count'})

# Sort by player count
versatility_stats = versatility_stats.sort_values('player_count', ascending=False)

versatility_stats

### Versatility Type Characteristics

Based on our analysis, we can characterize the four versatility types as follows:

1. **Stable Versatile**:
   - Consistently reliable performance across different team styles
   - Highly adaptable to different systems
   - Ideal for teams in transition or with evolving playing styles
   - Examples: [Examples from your data]

2. **Volatile Versatile**:
   - Variable performance but can adapt to different team styles
   - High ceiling across multiple systems
   - Valuable for teams that employ multiple styles within games
   - Examples: [Examples from your data]

3. **Stable Specialist**:
   - Consistently reliable performance but only in specific team styles
   - Limited adaptability to different systems
   - Valuable for teams with a well-defined, consistent playing style
   - Examples: [Examples from your data]

4. **Volatile Specialist**:
   - Variable performance and limited adaptability
   - Only effective in very specific contexts or systems
   - Highest risk profile for roster construction
   - Examples: [Examples from your data]

In [ ]:
# Find examples of players in each versatility type
versatility_examples = {}

for vtype in player_full_analysis['versatility_type'].unique():
    # Get players with this versatility type
    vtype_players = player_full_analysis[player_full_analysis['versatility_type'] == vtype]
    
    # Sort by points
    examples = vtype_players.nlargest(5, 'avg_pts')
    versatility_examples[vtype] = examples[['player_name', 'avg_pts', 'avg_plus_minus', 'system_stability', 'adaptability_score']]

# Display examples for each versatility type
for vtype, examples in versatility_examples.items():
    print(f"\n{vtype} Examples:")
    print(examples)

## Style Preference Vector

Let's extract player preferences for different playing styles.

In [ ]:
# Create style preference vectors for each player
player_style_preferences = []

for player_id in player_style_performance['Player_ID'].unique():
    player_data = player_style_performance[player_style_performance['Player_ID'] == player_id]
    player_name = player_data['PlayerName'].iloc[0]
    
    # Create preference vector based on plus/minus
    preferences = {}
    for _, row in player_data.iterrows():
        style = row['style_name']
        pm = row['PLUS_MINUS_mean']
        preferences[style] = pm
    
    # Normalize preferences to sum to 1
    min_pm = min(preferences.values())
    # Shift values to make them all positive
    if min_pm < 0:
        for style in preferences:
            preferences[style] -= min_pm - 1  # Ensure all values are positive
    
    # Calculate sum for normalization
    pref_sum = sum(preferences.values())
    if pref_sum > 0:
        for style in preferences:
            preferences[style] /= pref_sum
    
    # Determine best style
    best_style = max(preferences, key=preferences.get)
    
    # Add to results
    player_style_preferences.append({
        'player_id': player_id,
        'player_name': player_name,
        'preferences': preferences,
        'best_style': best_style
    })

# Display sample preference vectors
for i in range(min(5, len(player_style_preferences))):
    player = player_style_preferences[i]
    print(f"{player['player_name']} Style Preferences:")
    for style, pref in player['preferences'].items():
        print(f"  {style}: {pref:.3f}")
    print(f"  Best Style: {player['best_style']}\n")

In [ ]:
# Visualize style preferences for sample players
sample_players = [player_style_preferences[i]['player_name'] for i in range(min(3, len(player_style_preferences)))]

# Create a figure with subplots
fig, axes = plt.subplots(len(sample_players), 1, figsize=(10, 4 * len(sample_players)))

# Plot style preferences for each player
for i, player_name in enumerate(sample_players):
    # Get player preferences
    player_prefs = next(p for p in player_style_preferences if p['player_name'] == player_name)
    
    # Sort styles by preference
    sorted_styles = sorted(player_prefs['preferences'].items(), key=lambda x: x[1], reverse=True)
    styles = [s[0] for s in sorted_styles]
    prefs = [s[1] for s in sorted_styles]
    
    # Plot preferences
    if len(sample_players) > 1:
        ax = axes[i]
    else:
        ax = axes
    
    ax.bar(styles, prefs, color='skyblue')
    ax.set_title(f"{player_name}: Style Preferences", fontsize=12)
    ax.set_ylabel('Preference Score', fontsize=10)
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Create a dataframe with style preferences
style_names = list(player_style_preferences[0]['preferences'].keys())
player_pref_df = []

for player in player_style_preferences:
    player_row = {
        'player_id': player['player_id'],
        'player_name': player['player_name'],
        'best_style': player['best_style']
    }
    
    # Add preference for each style
    for style in style_names:
        player_row[f'pref_{style}'] = player['preferences'].get(style, 0)
    
    player_pref_df.append(player_row)

# Convert to dataframe
player_pref_df = pd.DataFrame(player_pref_df)

# Display the preference dataframe
player_pref_df.head()

In [ ]:
# Count players by best style
style_counts = player_pref_df['best_style'].value_counts().reset_index()
style_counts.columns = ['Style', 'Count']
style_counts['Percentage'] = style_counts['Count'] / len(player_pref_df) * 100

# Visualize style distribution
plt.figure(figsize=(10, 6))
plt.bar(style_counts['Style'], style_counts['Count'], color='skyblue')
plt.xlabel('Best Style', fontsize=12)
plt.ylabel('Number of Players', fontsize=12)
plt.title('Player Style Preferences Distribution', fontsize=14)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Interpreting Style Preferences

Our analysis of player style preferences reveals several interesting patterns:

1. **Style Distribution**: [Observations about the distribution of player preferences based on your data]
2. **Player Types and Preferences**: [Observations about how different player types prefer different styles based on your data]
3. **Preference Strength**: [Observations about how strongly players prefer their best style based on your data]

These insights can help teams identify players who are likely to thrive in their system and avoid players whose style preferences are incompatible with the team's approach.

## Team Composition Optimization

Let's analyze how to optimize team composition based on player stability profiles and style preferences.

In [ ]:
# Merge player preferences with full analysis
player_complete = pd.merge(
    player_full_analysis,
    player_pref_df[['player_id', 'player_name', 'best_style']],
    on=['player_id', 'player_name'],
    how='inner'
)

# Display the complete player analysis
player_complete.head()

In [ ]:
# Generate optimal lineups for each style
optimal_lineups = {}

for style in style_names:
    # Filter players who prefer this style
    style_players = player_complete[player_complete['best_style'] == style]
    
    # Ensure we have enough players
    if len(style_players) >= 5:
        # Select top players by plus/minus
        top_players = style_players.nlargest(5, 'avg_plus_minus')
        
        # Create lineup
        optimal_lineups[style] = top_players[['player_name', 'avg_pts', 'avg_plus_minus', 'system_stability', 'adaptability_score', 'versatility_type']]

# Display optimal lineups
for style, lineup in optimal_lineups.items():
    print(f"\nOptimal Lineup for {style}:")
    print(lineup)

In [ ]:
# Analyze stability balance in optimal lineups
lineup_stability = {}

for style, lineup in optimal_lineups.items():
    # Calculate average stability and adaptability
    avg_stability = lineup['system_stability'].mean()
    avg_adaptability = lineup['adaptability_score'].mean()
    
    # Count players by versatility type
    type_counts = lineup['versatility_type'].value_counts().to_dict()
    
    # Add to results
    lineup_stability[style] = {
        'avg_stability': avg_stability,
        'avg_adaptability': avg_adaptability,
        'type_counts': type_counts
    }

# Display lineup stability analysis
for style, stats in lineup_stability.items():
    print(f"\n{style} Lineup Stability Analysis:")
    print(f"  Average Stability: {stats['avg_stability']:.2f}")
    print(f"  Average Adaptability: {stats['avg_adaptability']:.2f}")
    print("  Versatility Type Distribution:")
    for vtype, count in stats['type_counts'].items():
        print(f"    {vtype}: {count}")

### Optimal Lineup Composition

Based on our analysis, we can make several recommendations for optimal lineup composition:

1. **Style-Specific Lineups**: [Recommendations for each style based on your data]
2. **Stability Balance**: [Recommendations for balancing stability in lineups based on your data]
3. **Versatility Mix**: [Recommendations for mixing versatility types based on your data]

These recommendations can help teams construct lineups that maximize performance within their preferred playing style while maintaining an appropriate balance of stability and adaptability.

## Player Cards

Let's create comprehensive player fit visualizations.

In [ ]:
# Create player fit cards for sample players
sample_players = player_complete['player_name'].iloc[:3].tolist()

# Create player fit cards
for player_name in sample_players:
    # Get player data
    player_data = player_complete[player_complete['player_name'] == player_name].iloc[0]
    player_id = player_data['player_id']
    
    # Get style performance data
    style_data = player_style_performance[player_style_performance['Player_ID'] == player_id]
    
    # Create figure
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Plot 1: Style performance (points)
    style_data_sorted = style_data.sort_values('PTS_mean', ascending=False)
    axes[0, 0].bar(style_data_sorted['style_name'], style_data_sorted['PTS_mean'], yerr=style_data_sorted['PTS_std'], 
                  capsize=5, color='skyblue')
    axes[0, 0].set_title('Points by Style', fontsize=12)
    axes[0, 0].set_ylabel('Points Per Game', fontsize=10)
    axes[0, 0].tick_params(axis='x', rotation=45)
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Style performance (plus/minus)
    style_data_sorted = style_data.sort_values('PLUS_MINUS_mean', ascending=False)
    axes[0, 1].bar(style_data_sorted['style_name'], style_data_sorted['PLUS_MINUS_mean'], yerr=style_data_sorted['PLUS_MINUS_std'], 
                  capsize=5, color='lightgreen')
    axes[0, 1].set_title('Plus/Minus by Style', fontsize=12)
    axes[0, 1].set_ylabel('Plus/Minus', fontsize=10)
    axes[0, 1].tick_params(axis='x', rotation=45)
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].axhline(y=0, color='gray', linestyle='--')
    
    # Plot 3: Style preferences
    player_prefs = next(p for p in player_style_preferences if p['player_name'] == player_name)
    sorted_styles = sorted(player_prefs['preferences'].items(), key=lambda x: x[1], reverse=True)
    styles = [s[0] for s in sorted_styles]
    prefs = [s[1] for s in sorted_styles]
    
    axes[1, 0].bar(styles, prefs, color='salmon')
    axes[1, 0].set_title('Style Preferences', fontsize=12)
    axes[1, 0].set_ylabel('Preference Score', fontsize=10)
    axes[1, 0].tick_params(axis='x', rotation=45)
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: Player metrics
    metrics = ['system_stability', 'adaptability_score', 'avg_pts', 'avg_plus_minus']
    metric_values = [player_data[m] for m in metrics]
    metric_labels = ['Stability', 'Adaptability', 'Points', 'Plus/Minus']
    
    # Normalize values for visualization
    max_values = {'system_stability': 2, 'adaptability_score': 10, 'avg_pts': 30, 'avg_plus_minus': 10}
    normalized_values = [metric_values[i] / max_values[metrics[i]] for i in range(len(metrics))]
    
    axes[1, 1].bar(metric_labels, normalized_values, color=['blue', 'green', 'orange', 'purple'])
    axes[1, 1].set_title('Player Metrics (Normalized)', fontsize=12)
    axes[1, 1].set_ylim(0, 1)
    axes[1, 1].grid(True, alpha=0.3)
    
    # Add actual values as text
    for i, v in enumerate(metric_values):
        axes[1, 1].text(i, normalized_values[i] + 0.05, f"{v:.2f}", ha='center')
    
    # Add player info
    plt.suptitle(f"{player_name} - {player_data['versatility_type']}\nBest Style: {player_data['best_style']}", fontsize=16)
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.show()

## Fit Improvement Simulation

Let's simulate how players might perform in different systems and identify potential trade targets based on style fit.

In [ ]:
# Simulate player performance in different systems
# For this simulation, we'll use the player's performance across different styles
# to estimate how they would perform if traded to a team with a different style

# Select a sample team
sample_team_id = team_styles['Team_ID'].iloc[0]
sample_team_name = team_styles[team_styles['Team_ID'] == sample_team_id]['TeamName'].iloc[0]
sample_team_style = team_styles[team_styles['Team_ID'] == sample_team_id]['style_name'].iloc[0]

print(f"Sample Team: {sample_team_name} (Style: {sample_team_style})")

# Identify potential trade targets
potential_targets = []

for player_id in player_style_performance['Player_ID'].unique():
    player_data = player_style_performance[player_style_performance['Player_ID'] == player_id]
    player_name = player_data['PlayerName'].iloc[0]
    
    # Check if player has played in the sample team's style
    if sample_team_style in player_data['style_name'].values:
        # Get player's performance in this style
        style_performance = player_data[player_data['style_name'] == sample_team_style].iloc[0]
        
        # Get player's average performance across all styles
        avg_performance = player_data.agg({
            'PTS_mean': 'mean',
            'PLUS_MINUS_mean': 'mean'
        })
        
        # Calculate performance difference
        pts_diff = style_performance['PTS_mean'] - avg_performance['PTS_mean']
        pm_diff = style_performance['PLUS_MINUS_mean'] - avg_performance['PLUS_MINUS_mean']
        
        # Calculate fit score
        fit_score = pts_diff + pm_diff * 2  # Weight plus/minus more heavily
        
        # Add to potential targets if fit score is positive
        if fit_score > 0:
            potential_targets.append({
                'player_id': player_id,
                'player_name': player_name,
                'pts_in_style': style_performance['PTS_mean'],
                'pm_in_style': style_performance['PLUS_MINUS_mean'],
                'avg_pts': avg_performance['PTS_mean'],
                'avg_pm': avg_performance['PLUS_MINUS_mean'],
                'pts_diff': pts_diff,
                'pm_diff': pm_diff,
                'fit_score': fit_score
            })

# Convert to dataframe
target_df = pd.DataFrame(potential_targets)

# Sort by fit score
target_df = target_df.sort_values('fit_score', ascending=False)

# Display top trade targets
print(f"\nTop Trade Targets for {sample_team_name}:")
print(target_df.head(10))

In [ ]:
# Visualize potential performance improvement for top targets
top_targets = target_df.head(5)

plt.figure(figsize=(12, 6))

# Plot points improvement
x = np.arange(len(top_targets))
width = 0.35

plt.bar(x - width/2, top_targets['avg_pts'], width, label='Average Points', color='skyblue')
plt.bar(x + width/2, top_targets['pts_in_style'], width, label=f'Points in {sample_team_style}', color='salmon')

plt.xlabel('Player', fontsize=12)
plt.ylabel('Points Per Game', fontsize=12)
plt.title(f'Potential Performance Improvement in {sample_team_name} System', fontsize=14)
plt.xticks(x, top_targets['player_name'], rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Trade Target Recommendations

Based on our fit improvement simulation, we can make the following recommendations for [sample_team_name]:

1. **Top Trade Targets**:
   - [List top trade targets based on your data]
   - These players are likely to perform significantly better in the [sample_team_style] system compared to their current average performance.

2. **Expected Improvement**:
   - [Player 1]: +[X] PPG, +[Y] Plus/Minus
   - [Player 2]: +[X] PPG, +[Y] Plus/Minus
   - [Player 3]: +[X] PPG, +[Y] Plus/Minus

3. **Fit Considerations**:
   - [Additional considerations based on your data]

These recommendations demonstrate the potential value of considering style fit in player acquisition decisions.

## Save Player Fit Analysis for Subsequent Analysis

Let's save our player fit analysis for use in subsequent notebooks.

In [ ]:
# Save player fit data
player_complete.to_csv('../data/processed/player_team_fit.csv', index=False)
print(f"Saved player team fit data to ../data/processed/player_team_fit.csv")

## Conclusion

In this notebook, we've analyzed how players perform across different team systems, quantified player adaptability, and identified optimal style environments for different player types. We've developed a comprehensive framework for evaluating player-team fit that can inform roster construction, player acquisition, and lineup optimization decisions.

Key accomplishments:
1. Calculated player performance metrics across different team styles
2. Quantified player adaptability to different systems
3. Identified optimal style environments for players
4. Classified players into versatility types based on stability and adaptability
5. Created player style preference vectors
6. Generated optimal lineups for each team style
7. Created comprehensive player fit visualizations
8. Simulated player performance in different systems

Our analysis demonstrates the importance of considering player-team fit in basketball operations decisions and provides a framework for maximizing player performance through optimal system matching. In the next notebook, we'll explore teammate network analysis to understand how players influence each other's performance.